# 8. Systematic uncertainties

In this exercise, we will analyse the data of PKS 2155-304 from the H.E.S.S. DL3-DR1, compute its average spectrum and compare two spectral hypotheses. Then, we will assess the systematic uncertainties on the spectrum by studying the effect of a bias on the energy scale.

For this tutorial, we will need a few extra python packages (such as `astroquery` to query Virtual Observatory services).

In [ ]:
# !mamba install -c conda-forge astroquery corner tqdm
#
# or:
#
# !mamba create -n cads-2024 -c conda-forge gammapy=1.2 ipykernel astroquery corner tqdm

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.style as style
style.use('tableau-colorblind10')

import corner
import numpy as np
import os

from IPython.display import display
from scipy.stats import chi2, norm
from tqdm import tqdm

from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.table import Table
from astroquery.simbad import Simbad

from gammapy.data import DataStore
from gammapy.datasets import Datasets, SpectrumDataset, SpectrumDatasetOnOff
from gammapy.estimators import FluxPointsEstimator
from gammapy.estimators.utils import resample_energy_edges
from gammapy.makers.utils import make_theta_squared_table
from gammapy.makers import (
    DatasetsMaker,
    SafeMaskMaker,
    SpectrumDatasetMaker,
    ReflectedRegionsBackgroundMaker,
)
from gammapy.maps import MapAxis, RegionGeom, WcsGeom
from gammapy.modeling import Fit, Parameter
from gammapy.modeling.models import (
    LogParabolaSpectralModel,
    PowerLawSpectralModel,
    SpectralModel,
    SkyModel,
)
from gammapy.utils import pbar
pbar.SHOW_PROGRESS_BAR = True
from gammapy.visualization import plot_spectrum_datasets_off_regions, plot_theta_squared_table

from regions import CircleSkyRegion

We first load the data for the H.E.S.S. DL3-DR1:

In [ ]:
data_store = DataStore.from_dir(XXX)

We set the properties of the source of interest. Bonus: we use Virtual Observatory services to query the source parameters.

In [ ]:
src = dict()
src['Name'] = 'PKS 2155-304'
src['Position'] = SkyCoord.from_name(src['Name'])

try:
    simbad = Simbad()
    simbad.add_votable_fields("z_value")
    query = simbad.query_object(src['Name'])
    src['Redshift'] = query["Z_VALUE"].data[0]
except NameError:
    src['Redshift'] = 0.116

We select a sub-sample of H.E.S.S. data acquired on our source:

In [ ]:
selection = dict(XXX)
selected_obs_table = data_store.obs_table.select_observations(selection)

observations = XXX

In [ ]:
exclusion_mask = XXX

# Data reduction

Let's perform a 1D analysis of the data.

In [ ]:
energy_axis = XXX
energy_axis_true = XXX

geom = XXX

dataset_empty = XXX
dataset_maker = XXX

bkg_maker = XXX
safe_mask_maker = XXX

In [ ]:
%%time

# Parallel version
makers = [dataset_maker, bkg_maker, safe_mask_maker]  # the order matters
datasets_maker = DatasetsMaker(makers, stack_datasets=False, n_jobs=6)
datasets = datasets_maker.run(dataset_empty, observations)

In [ ]:
fig, (ax_excess, ax_sqrt_ts) = plt.subplots(figsize=(10, 4), ncols=2, nrows=1)
ax_excess.plot(
    info_table["livetime"].to("h"),
    info_table["excess"],
    marker="o",
    ls="none",
)

ax_excess.set_title("Excess")
ax_excess.set_xlabel("Livetime [h]")
ax_excess.set_ylabel("Excess events")

ax_sqrt_ts.plot(
    info_table["livetime"].to("h"),
    info_table["sqrt_ts"],
    marker="o",
    ls="none",
)

ax_sqrt_ts.set_title("Sqrt(TS)")
ax_sqrt_ts.set_xlabel("Livetime [h]")
ax_sqrt_ts.set_ylabel("Sqrt(TS)")

plt.show()

In [ ]:
theta2_axis = MapAxis.from_bounds(0, 0.1, nbin=50, interp="lin", unit="deg2")

theta2_table = make_theta_squared_table(
    observations=observations,
    position=src['Position'],
    theta_squared_axis=theta2_axis,
)

on_region_radius = Angle("0.1 deg")

fig = plt.figure(figsize=(8, 5))
plot_theta_squared_table(theta2_table)
for ax in fig.get_axes():
    ax.axvline(on_region_radius.value**2, color="red", linestyle="--", label="ON region")
    ax.legend()
plt.show()

In [ ]:
dataset_stack = datasets.stack_reduce(name="hess")

# Fit stacked spectrum

We will fit the overall spectrum of PKS 2155-304, under two hypotheses:
* a simple power-law
* a log-parabola

We will judge which hypotheses reperesent best the data by comparing the fit statistics.

In [ ]:
reference_energy = 300. * u.Unit("GeV")

In [ ]:
dataset_stack_pl = dataset_stack.copy(name="hess")

spectral_model_pl = PowerLawSpectralModel(XXX)

source_pl = SkyModel(XXX)

dataset_stack_pl.models = source_pl
print(dataset_stack_pl)

In [ ]:
%%time

fit_pl = Fit()
result_pl = fit_pl.run(datasets=dataset_stack_pl)

# we make a copy here to compare it later
model_best_pl = source_pl.copy(name=src['Name'])

In [ ]:
model_best_pl

In [ ]:
dataset_stack_lp = dataset_stack.copy(name="hess")

spectral_model_lp = LogParabolaSpectralModel(XXX)

source_lp = SkyModel(XXX)

dataset_stack_lp.models = source_lp
print(dataset_stack_lp)

In [ ]:
%%time

fit_lp = Fit()
result_lp = fit_lp.run(datasets=dataset_stack_lp)

# we make a copy here to compare it later
model_best_lp = source_lp.copy(name=src['Name'])

In [ ]:
model_best_lp

In [ ]:
def sigma_lp_vs_pl(df=1):
    ts_pl = result_pl.total_stat
    ts_lp = result_lp.total_stat
    Delta_TS = ts_pl-ts_lp
    p_value = chi2.sf(Delta_TS, df=df)
    sigma = norm.isf(0.5*p_value)  # 0.5 only for 1-sided hypothesis on curvature parameter (which is constrained positive)
    return sigma

Assess which spectral model is to be preferred.

In [ ]:
XXX

# Compute flux points

In [ ]:
energy_edges = resample_energy_edges(dataset_stack, conditions={'sqrt_ts_min': 2.})

fpe = FluxPointsEstimator(XXX)
flux_points = fpe.run([dataset_stack])

# Spectral energy distribution

We will produce an SED with the best-fit model and results.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

XXX

# Systematics study

Now, we will define a custom spectral model by introducing a nuisance parameter on the energy scale, and study the systematic effet of a bias on this parameter.

In [ ]:
# Define some custom biased spectral models

class BiasedPowerLawSpectralModel(SpectralModel):
    tag = "BiasedPowerLawSpectralModel"
    amplitude = Parameter("amplitude", "1e-12 cm-2 s-1 TeV-1", min=0, is_norm=True)
    index = Parameter("index", 2.5, min=0)
    reference = Parameter("reference", "1 TeV", frozen=True)
    bias = Parameter("bias", 1., min=0.)
        
    @staticmethod
    def evaluate(energy, amplitude, index, reference, bias):
        energy = bias.value * energy
        pwl = PowerLawSpectralModel.evaluate(
            energy=energy,
            index=index,
            amplitude=amplitude,
            reference=reference,
        )
        return pwl

class BiasedLogParabolaSpectralModel(SpectralModel):
    tag = "BiasedLogParabolaSpectralModel"
    amplitude = Parameter("amplitude", "1e-12 cm-2 s-1 TeV-1", min=0, is_norm=True)
    alpha = Parameter("alpha", 2.5, min=0)
    beta = Parameter("beta", 0.5)
    reference = Parameter("reference", "1 TeV", frozen=True)
    bias = Parameter("bias", 1., min=0.)
        
    @staticmethod
    def evaluate(energy, amplitude, alpha, beta, reference, bias):
        energy = bias.value * energy
        logpwl = LogParabolaSpectralModel.evaluate(
            energy=energy,
            alpha=alpha,
            beta=beta,
            amplitude=amplitude,
            reference=reference,
        )
        return logpwl

In [ ]:
# Define the biased model, based on the best model shape obtained above (PL or LP)

spectral_model_biased = XXX

biased_model = SkyModel(XXX)
print(biased_model)

Now, we will simulate datasets injecting the biased model, assuming 10% of uncertainty on the energy scale.

In [ ]:
%%time

# Definition of reference dataset
reference_dataset = dataset_stack.copy(name="hess")

# Fake counts taking bias into account
simulated_biased_datasets = []
n_sim = 1000
energy_bias = 10./100.

results_biased = []

for i in tqdm(range(n_sim)):
    ds = reference_dataset
    # Random bias
    biased_model.spectral_model.bias.value = np.random.normal(XXX)
    
    # Set the model on the ON-OFF dataset using the *biased* model
    ds.XXX
    
    # Fake the dataset
    ds.XXX
    

    # Fit the simulated dataset
    fit_result = XXX

    if fit_result.success:
        par_dict = {}
        for par in fit_result.parameters.free_parameters:
            par_dict[par.name] = par.quantity
        results_biased.append(par_dict)
        
    simulated_biased_datasets.append(ds)

In [ ]:
fitted_params_with_energy_bias = Table(results_biased).to_pandas()
biased_mean = fitted_params_with_energy_bias.mean()
biased_uncertainty = fitted_params_with_energy_bias.std()

In [ ]:
for par in result.models[src['Name']].spectral_model.parameters.free_parameters:
    statistic_uncertainty = par.error * par.unit
    total_uncertainty = biased_uncertainty[par.name] * par.unit
    systematic_uncertainty = np.sqrt(np.abs(total_uncertainty**2 - statistic_uncertainty**2))
    print(f"{par.name} : {par.quantity.value:.3e} +/-\t"
          f" {statistic_uncertainty.value:.3e} (stat.) +/-\t"
          f" {systematic_uncertainty.value:.3e} (syst.) {par.quantity.unit}")

In [ ]:
fitted_params_with_energy_bias['amplitude'] *= 1e10

figure = corner.corner(fitted_params_with_energy_bias,
                       quantiles=[0.16, 0.5, 0.84],
                       show_titles=True,
                       title_kwargs={"fontsize": 12})

Let's plot the resulting SED with the effect of the uncertainties on the energy scale:

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

plot_kwargs = {
    "energy_bounds": [energy_edges[0], energy_edges[-1]],
    "sed_type": "e2dnde",
    "yunits": u.Unit("erg cm-2 s-1"),
    "ax": ax,
}

spec = XXX


for i, result_biased in enumerate(results_biased):
    XXX

Let's have a look at the likelihood profiles for the fitted parameters and their correlations.

In [ ]:
# Likelihood profiles

XXX

In [ ]:
# Parameters correlations

XXX